In [1]:
import os

import pandas as pd
from pandas import DataFrame
import numpy as np
from scipy.stats.mstats import winsorize

from Constants import Constants as const

In [2]:
pc3_df: DataFrame = pd.read_stata(os.path.join(const.DATA_PATH, 'fromZGY', 'firm_year_num_political_donar_3years.dta')).rename(
    columns={'recipient_ext_id': 'numDonation3year'}).drop(['index'], axis=1)
pc4_df: DataFrame = pd.read_stata(os.path.join(const.DATA_PATH, 'fromZGY', 'firm_year_num_political_donar_4year.dta')).rename(
    columns={'recipient_ext_id': 'numDonation4year'}).drop(['index'], axis=1)

In [7]:
pc3_avg = pc3_df.groupby([const.GVKEY])['numDonation3year'].mean()
pc4_avg = pc4_df.groupby([const.GVKEY])['numDonation4year'].mean()

In [9]:
pc3_avg_df: DataFrame = pc3_avg.reset_index(drop=False)
pc4_avg_df: DataFrame = pc4_avg.reset_index(drop=False)

In [10]:
reg_df: DataFrame = pd.read_stata(os.path.join(const.RESULT_PATH, '20241014_stock_act_reg_data_v2.dta'))
gvkey_series = reg_df[const.GVKEY].unique()

C:\Users\wangy\AppData\Local\Temp\ipykernel_20864\88727916.py:1: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  reg_df: DataFrame = pd.read_stata(os.path.join(const.RESULT_PATH, '20241014_stock_act_reg_data_v2.dta'))


In [13]:
# Update the function to handle numpy.ndarray as input
def ensure_gvkeys_with_ndarray(df, gvkey_array, num_col_name):
    # Convert the numpy array to a set for quick lookup
    gvkey_set = set(gvkey_array)
    # Identify missing gvkeys
    existing_gvkeys = set(df['gvkey'])
    missing_gvkeys = gvkey_set - existing_gvkeys
    # Create a DataFrame with missing gvkeys
    missing_rows = pd.DataFrame({
        'gvkey': list(missing_gvkeys),
        num_col_name: [None] * len(missing_gvkeys)
    })
    # Concatenate the original DataFrame with missing rows and sort by gvkey
    updated_df = pd.concat([df, missing_rows], ignore_index=True)
    updated_df = updated_df.sort_values('gvkey').reset_index(drop=True)
    return updated_df

In [14]:
# Update pc3_avg_df and pc4_avg_df
pc3_avg_df2 = ensure_gvkeys_with_ndarray(pc3_avg_df, gvkey_series, 'numDonation3year')
pc4_avg_df2 = ensure_gvkeys_with_ndarray(pc4_avg_df, gvkey_series, 'numDonation4year')


C:\Users\wangy\AppData\Local\Temp\ipykernel_20864\1236285319.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  updated_df = pd.concat([df, missing_rows], ignore_index=True)


In [16]:
pc_avg_df: DataFrame = pc3_avg_df2.merge(pc4_avg_df2, on=[const.GVKEY])
pc_avg_df.shape

(1837, 3)

In [17]:
for key in ['numDonation3year', 'numDonation4year']:
    pc_avg_df[key] = pc_avg_df[key].fillna(0)

In [20]:
pc_avg_df['highDonation3Year'] = (pc_avg_df['numDonation3year'] > pc_avg_df['numDonation3year'].median()).astype(int)
pc_avg_df['highDonation4Year'] = (pc_avg_df['numDonation4year'] > pc_avg_df['numDonation4year'].median()).astype(int)

In [22]:
reg_df2: DataFrame = reg_df.merge(pc_avg_df, on=[const.GVKEY], how='left')
reg_df2.to_stata(os.path.join(const.RESULT_PATH, '20241212_stock_act_reg_data.dta'), write_index=False, version=119)

# Append Government Contract data

In [2]:
reg_df: DataFrame = pd.read_stata(os.path.join(const.RESULT_PATH, '20241212_stock_act_reg_data.dta'))
gov_contract_df: DataFrame = pd.read_csv(os.path.join(const.DATA_PATH, 'fromZGY', 'gov_characters_data.csv')).drop(['tic'], axis=1)

In [3]:
gov_contract_df.rename(columns=lambda x: '{}_num'.format(x) if x.endswith('gov') else x, inplace=True)

In [4]:
gov_contract_df

,fiscal_year,fed_gov_num,gov_num,state_gov_num,loc_gov_num,gvkey
0,2008.0,0.0,0.0,0.0,0.0,151832
1,2009.0,0.0,0.0,0.0,0.0,151832
2,2010.0,0.0,0.0,0.0,0.0,151832
3,2011.0,0.0,0.0,0.0,0.0,151832
4,2012.0,0.0,0.0,0.0,0.0,151832
...,...,...,...,...,...,...
85811,2011.0,0.0,0.0,0.0,0.0,30165
85812,2012.0,0.0,0.0,0.0,0.0,30165
85813,2013.0,0.0,0.0,0.0,0.0,30165
85814,2014.0,0.0,0.0,0.0,0.0,30165


In [10]:
reg_df_gov_num = reg_df.merge(gov_contract_df, on=[const.GVKEY, const.YEAR], how='left')
for key in gov_contract_df.keys():
    if key not in {const.GVKEY, const.YEAR}:
        reg_df_gov_num[key].fillna(0, inplace=True)
        reg_df_gov_num[key.replace('num', 'dummy')] = reg_df_gov_num[key].apply(lambda x: int(x > 0))

C:\Users\wangy\AppData\Local\Temp\ipykernel_28288\2904554935.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  reg_df_gov_num[key].fillna(0, inplace=True)
C:\Users\wangy\AppData\Local\Temp\ipykernel_28288\2904554935.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example,

In [11]:
reg_df_gov_num.describe()

,index,gvkey,fiscal_year,mf_indicator,frequency,log_frequency,MajorGovCustomer,post,log_market_value,lev,...,highDonation3Year,highDonation4Year,fed_gov_num,gov_num,state_gov_num,loc_gov_num,fed_gov_dummy,gov_dummy,state_gov_dummy,loc_gov_dummy
count,12074.00000,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,...,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000
mean,6036.50000,81220.205317,2011.546960,0.363508,4.817459,0.868412,0.153553,0.508945,6.517410,0.181687,...,0.439043,0.465629,0.263541,0.295511,0.023025,0.008945,0.152725,0.160593,0.016316,0.006957
min,0.00000,1004.000000,2008.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.786519,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3018.25000,12142.000000,2010.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.012064,0.000930,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,6036.50000,62592.000000,2012.000000,0.000000,0.000000,0.000000,0.000000,1.000000,6.520895,0.148584,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,9054.75000,156153.000000,2014.000000,1.000000,7.000000,2.079442,0.000000,1.000000,7.976261,0.264176,...,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,12073.00000,296753.000000,2015.000000,1.000000,36.000000,3.610918,1.000000,1.000000,11.561005,17.825450,...,1.000000,1.000000,19.000000,19.000000,10.000000,3.000000,1.000000,1.000000,1.000000,1.000000
std,3485.60791,73644.923742,2.279738,0.481029,8.551352,1.245300,0.360535,0.499941,2.163776,0.273871,...,0.496291,0.498838,0.862415,0.934783,0.219107,0.114772,0.359737,0.367170,0.126693,0.083122


In [12]:
reg_df_gov_num.to_stata(os.path.join(const.RESULT_PATH, '20241223_stock_act_reg_data.dta'), write_index=False, version=119)